In [1]:
import pandas as pd
from unidecode import unidecode

In [2]:
all_data = pd.read_excel('citsmart_124565_alunos_introducao_computacao_tratado.xlsx')

In [3]:
# comentar se for usar a base inteira
all_data = all_data[(all_data['mencao_disciplina'] == 'SS') | (all_data['mencao_disciplina'] == 'MS') | (all_data['mencao_disciplina'] == 'MM') | (all_data['mencao_disciplina'] == 'MI') | (all_data['mencao_disciplina'] == 'II') | (all_data['mencao_disciplina'] == 'SR')]

In [4]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47676 entries, 0 to 52785
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          47676 non-null  int64  
 1   Aluno                               47676 non-null  object 
 2   genero                              47676 non-null  object 
 3   nascimento                          47676 non-null  object 
 4   endereco                            47676 non-null  object 
 5   estado_nascimento                   47676 non-null  object 
 6   sistema_cotas                       47676 non-null  object 
 7   cota                                47676 non-null  object 
 8   Escola                              47676 non-null  object 
 9   chamada_ingressou_UnB               47676 non-null  object 
 10  curso                               47676 non-null  object 
 11  periodo_ingresso_unb                47676

In [5]:
# Connect to data base
import mysql.connector as mysql
from mysql.connector import Error
try:
    db = mysql.connect(
        host = "localhost",
        #database='mestrado2'
        database='mestradoNOTA',
        user = "root",
        passwd = "admin123",
        auth_plugin='mysql_native_password'
    )
    
    if db.is_connected():
        cursor = db.cursor(buffered=True)
#         cursor = db.cursor()
except Error as e:
    print("Error while connecting to MySQL", e)
    

In [6]:
cursor.execute("DROP TABLE IF EXISTS student_course_subject")
cursor.execute("DROP TABLE IF EXISTS student_courses")
cursor.execute("DROP TABLE IF EXISTS students")
cursor.execute("DROP TABLE IF EXISTS subjects")


# TABELA MATERIAS
# Código.da.disciplina	Nome.da.disciplina	Créditos.disciplina
cursor.execute("CREATE TABLE subjects (id INT AUTO_INCREMENT PRIMARY KEY, code VARCHAR(255) NOT NULL, name VARCHAR(255) NOT NULL, credits int(10) NOT NULL)")


# TABELA ALUNOS
# 	ID.Aluno	Sexo	UF.de.Nascimento	Cotista	
# Tipo.de.Escola	Raça	Periodo.de.Ingresso.na.Unb	Forma.de.Ingresso.na.UnB
cursor.execute("CREATE TABLE students (id VARCHAR(255) NOT NULL PRIMARY KEY, gender VARCHAR(255) NOT NULL, birth_date VARCHAR(255) NOT NULL, uf_birth VARCHAR(10) NOT NULL, adress VARCHAR(255) NOT NULL, isquotaholder VARCHAR(10) NOT NULL, quotaholder_type VARCHAR(255), school_type VARCHAR(255) NOT NULL, university_entrance_period VARCHAR(255) NOT NULL, university_entrance_form VARCHAR(255), university_entrance_call VARCHAR(255) NOT NULL)")

# TABELA ALUNOS_CURSO
# ID, ID.Aluno, Curso, Opção, Periodo.de.ingresso.na.opção, Período.de.saída.da.opção, 
# Forma.de.saída.da.opção, Min..Créd..Formatura	
# Créditos.cursados.no.total	Créditos.integralizados.no.total	Créditos.a.integralizar.no.total
cursor.execute("CREATE TABLE student_courses (id INT AUTO_INCREMENT PRIMARY KEY, course VARCHAR(255) NOT NULL, course_entrance_period VARCHAR(255) NOT NULL, leaving_form VARCHAR(255) NOT NULL, leaving_period VARCHAR(255) NOT NULL, minimum_graduation_credit VARCHAR(255) NOT NULL, id_student VARCHAR(255) NOT NULL, FOREIGN KEY(id_student) REFERENCES students(id) ON DELETE CASCADE)")

# TABELA_ALUNOS_MATERIA_CURSO
# Ano.e.Semestre	Média.do.período	Ano.e.Semestre
# Média.do.período}	ID.Aluno	(Código.da.disciplina)	(ID.Aluno.Curso)
cursor.execute("CREATE TABLE student_course_subject(id INT AUTO_INCREMENT PRIMARY KEY, score VARCHAR(255) NOT NULL, num_score INT NOT NULL, average_in_semester VARCHAR(255) NOT NULL, student_semester int NOT NULL, semester_year VARCHAR(255) NOT NULL, total_credits_semester int NOT NULL, approved_total_credits_semester int NOT NULL, id_student VARCHAR(255) NOT NULL, id_student_course INT NOT NULL, id_subject INT NOT NULL, FOREIGN KEY(id_student) REFERENCES students(id) ON DELETE CASCADE, FOREIGN KEY(id_student_course) REFERENCES student_courses(id) ON DELETE CASCADE, FOREIGN KEY(id_subject) REFERENCES subjects(id) ON DELETE CASCADE)")


cursor.execute("SHOW TABLES")

for x in cursor:
  print(x)


('student_course_subject',)
('student_courses',)
('students',)
('subjects',)


## Student

In [7]:
values_students = all_data[['id_student', 'genero', 'nascimento', 'endereco', 'estado_nascimento', 'sistema_cotas', 'cota', 'Escola', 'periodo_ingresso_unb', 'forma_ingresso_unb', 'chamada_ingressou_UnB']]
uniqueValuesStudents = values_students.drop_duplicates()

In [8]:
print(uniqueValuesStudents['id_student'].count())
print(values_students['id_student'].count())
print(all_data['id_student'].count())

8052
47676
47676


## Student_courses

In [9]:
values_student_courses = all_data[['id_student', 'curso', 'periodo_ingresso_curso', 'periodo_saida_curso', 'forma_saida_curso', 'min_cred_para_formatura']]
uniqueValuesStudentCourses = values_student_courses.drop_duplicates()

In [10]:
print(uniqueValuesStudentCourses['id_student'].count())

8246


In [11]:
uniqueValuesStudentCourses[['curso', 'min_cred_para_formatura']].drop_duplicates()

,curso,min_cred_para_formatura
0,Ciencia da Computacao,240
1735,Ciencia da Computacao,214
17392,Computacao,180
23889,Computacao,214
28861,Engenharia Civil,262
29778,Engenharia de Computacao,252
41653,Engenharia Eletrica,262
41860,Engenharia Eletrica,248
42560,Engenharia Florestal,258
43322,Engenharia Mecanica,262


In [12]:
def arrumar_curso(row, curso, min_creditos_antigo, min_creditos_novo):
    if((row['curso'] == curso) & (row['min_cred_para_formatura'] == min_creditos_antigo)):
        found2 = uniqueValuesStudentCourses[(uniqueValuesStudentCourses['id_student'] == row['id_student']) & 
            (uniqueValuesStudentCourses['curso'] == curso) & 
            (uniqueValuesStudentCourses['min_cred_para_formatura'] == min_creditos_novo) & 
            (uniqueValuesStudentCourses['forma_saida_curso'] == row['forma_saida_curso'])]
        if(len(found2) > 0):
            return 'Mudança de curriculo'
    return row['forma_saida_curso']

In [13]:
uniqueValuesStudentCourses['forma_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_curso(x, 'Ciencia da Computacao', 240, 214), axis=1)
uniqueValuesStudentCourses['forma_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_curso(x, 'Computacao', 214, 180), axis=1)
uniqueValuesStudentCourses['forma_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_curso(x, 'Engenharia Eletrica', 262, 248), axis=1)
uniqueValuesStudentCourses['forma_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_curso(x, 'Estatistica', 184, 200), axis=1)
uniqueValuesStudentCourses['forma_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_curso(x, 'Matematica', 168, 188), axis=1)

<ipython-input-13-22c408342484>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqueValuesStudentCourses['forma_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_curso(x, 'Ciencia da Computacao', 240, 214), axis=1)
<ipython-input-13-22c408342484>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqueValuesStudentCourses['forma_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_curso(x, 'Computacao', 214, 180), axis=1)
<ipython-input-13-22c408342484>:3: SettingWithCopyWar

In [14]:
def arrumar_periodo_de_saida(row, curso, ano, creditos):
    if((row['forma_saida_curso'] == 'Mudança de curriculo') & (row['min_cred_para_formatura'] == creditos) & (row['curso'] == curso)):
        return ano
    return row['periodo_saida_curso']

In [15]:
uniqueValuesStudentCourses['periodo_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_periodo_de_saida(x, 'Ciencia da Computacao', 20051, 240), axis=1)
uniqueValuesStudentCourses['periodo_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_periodo_de_saida(x, 'Computacao', 20051, 180), axis=1)
uniqueValuesStudentCourses['periodo_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_periodo_de_saida(x, 'Engenharia Eletrica', 20051, 248), axis=1)
uniqueValuesStudentCourses['periodo_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_periodo_de_saida(x, 'Estatistica', 20051, 200), axis=1)
uniqueValuesStudentCourses['periodo_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_periodo_de_saida(x, 'Matematica', 20051, 188), axis=1)


<ipython-input-15-63d702393bad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqueValuesStudentCourses['periodo_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_periodo_de_saida(x, 'Ciencia da Computacao', 20051, 240), axis=1)
<ipython-input-15-63d702393bad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqueValuesStudentCourses['periodo_saida_curso'] = uniqueValuesStudentCourses.apply(lambda x: arrumar_periodo_de_saida(x, 'Computacao', 20051, 180), axis=1)
<ipython-input-15-63d70

### Subjects

In [16]:
values_subjects = all_data[['codigo_disciplina', 'nome_disciplina', 'creditos_disciplina']]
uniqueValuesSubjects = values_subjects.drop_duplicates()

In [17]:
print(uniqueValuesSubjects['codigo_disciplina'].count())
print(len(uniqueValuesSubjects['codigo_disciplina'].unique().tolist()))
print(len(uniqueValuesSubjects['nome_disciplina'].unique().tolist()))
print(len(uniqueValuesSubjects[['nome_disciplina', 'creditos_disciplina']].drop_duplicates()))
# print(uniqueValuesSubjects['nome_disciplina'].count())


134
132
132
133


In [18]:
# teste = all_data[(all_data['mencao_disciplina'] == 'SS') | (all_data['mencao_disciplina'] == 'MS') | (all_data['mencao_disciplina'] == 'MM') | (all_data['mencao_disciplina'] == 'MI') | (all_data['mencao_disciplina'] == 'II') | (all_data['mencao_disciplina'] == 'SR')]
# values_subjects = teste[['codigo_disciplina', 'nome_disciplina', 'creditos_disciplina']]
# uniqueValuesSubjects = values_subjects.drop_duplicates()
# print(len(teste))
# print(len(all_data))


In [19]:
# print(uniqueValuesSubjects['codigo_disciplina'].count())
# print(len(uniqueValuesSubjects['codigo_disciplina'].unique().tolist()))
# print(len(uniqueValuesSubjects['nome_disciplina'].unique().tolist()))
# print(len(uniqueValuesSubjects[['nome_disciplina', 'creditos_disciplina']].drop_duplicates()))
# # print(uniqueValuesSubjects['nome_disciplina'].count())


## Student_course_subject

In [20]:
values_student_course_subject = all_data

In [21]:
dic = {
    'SS': 5,
    'MS': 4,
    'MM': 3,
    'MI': 2,
    'II': 1,
    'SR': 0,
    'TJ': -1,
    'TR': -1,
    'CC': -1,
    'AP': -1,
    'DP': -1,
}
values_student_course_subject['num_mencao_disciplina'] = values_student_course_subject['mencao_disciplina'].apply(lambda x: dic[x])

### criar campo semestre p representar em qual semestre cada aluno fez cada materia

In [22]:
def create_dic(row):
    aux = values_student_course_subject[values_student_course_subject['id_student'] == row['id_student']]
    
    keys_list = aux['periodo_cursou_disciplina'].drop_duplicates().to_list()
#     print(keys_list)
#     print(keys_list.sort())
    values_list = range(1, 1 + len(keys_list))
    zip_iterator = zip(keys_list, values_list)
    
    a_dictionary = dict(zip_iterator)
    
    return a_dictionary[row['periodo_cursou_disciplina']]

In [23]:
values_student_course_subject['semester'] = values_student_course_subject.apply(lambda x: create_dic(x), axis=1)

# Inserção dos dados

In [24]:
# Tabela Alunos
for i,row in uniqueValuesStudents.iterrows():
    sql = "INSERT INTO students (id, gender, birth_date, adress, uf_birth, isquotaholder, quotaholder_type, school_type, university_entrance_period, university_entrance_form, university_entrance_call) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    db.commit()


In [25]:
# Tabela Matérias
for i,row in uniqueValuesSubjects.iterrows():
    sql = "INSERT INTO subjects (code, name, credits) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    db.commit()


In [26]:
# Tabela Aluno_cursos
for i,row in uniqueValuesStudentCourses.iterrows():
    sql = "INSERT INTO student_courses(id_student, course, course_entrance_period, leaving_period, leaving_form, minimum_graduation_credit) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    db.commit()

In [27]:
# Tabela Aluno_curso_materia
for i,row in all_data.iterrows():
#     print(tuple(row))
    select_idcurso = "SELECT id FROM student_courses WHERE id_student = %s AND course = %s AND course_entrance_period = %s AND minimum_graduation_credit = %s"
    cursor.execute(select_idcurso, (row['id_student'], row['curso'], row['periodo_ingresso_curso'], row['min_cred_para_formatura']) )

    ## fetching all usernames from the 'cursor' object
    id_student_courses = cursor.fetchall()
    if len(id_student_courses) > 1:
        print('Erro', id_student_courses)
        exit()

    id_student_courses = id_student_courses[0]

    select_idsubject = "SELECT id from subjects WHERE code = %s AND name = %s AND credits = %s"
    cursor.execute(select_idsubject, (row['codigo_disciplina'], row['nome_disciplina'],  row['creditos_disciplina']) )
    
    id_subjects = cursor.fetchall()
    if len(id_subjects) > 1:
        print('Erro', id_subjects)
        exit()

    id_subjects = id_subjects[0]

    sql = "INSERT INTO student_course_subject (id_student, id_student_course, id_subject, score, num_score, average_in_semester, student_semester, semester_year, total_credits_semester, approved_total_credits_semester) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, (row['id_student'], id_student_courses[0], id_subjects[0], row['mencao_disciplina'], row['num_mencao_disciplina'], row['media_semestre_aluno'], row['semester'], row['periodo_cursou_disciplina'], row['creditos_no_periodo'], row['creditos_creditos_aprovado_periodo']))
    db.commit()

In [28]:
db.close()